In [85]:
# Copyright 2020 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Keras text dataset generation utilities."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np


from tensorflow.python.data.ops import dataset_ops
from tensorflow.python.keras.preprocessing import dataset_utils
from tensorflow.python.ops import io_ops
from tensorflow.python.ops import string_ops
from tensorflow.python.util.tf_export import keras_export

directory=r"C:/Users/thinb/Desktop/article-evaluator/results/train"


@keras_export('keras.preprocessing.text_dataset_from_directory', v1=[])
def text_dataset_from_directory(directory,
                                labels='inferred',
                                label_mode='binary',
                                class_names=None,
                                batch_size=32,
                                max_length=None,
                                shuffle=False,
                                seed=None,
                                validation_split=0.2,
                                subset='validation',
                                follow_links=False):
  """Generates a `tf.data.Dataset` from text files in a directory.

  If your directory structure is:

  ```
  main_directory/
  ...class_a/
  ......a_text_1.txt
  ......a_text_2.txt
  ...class_b/
  ......b_text_1.txt
  ......b_text_2.txt
  ```

  Then calling `text_dataset_from_directory(main_directory, labels='inferred')`
  will return a `tf.data.Dataset` that yields batches of texts from
  the subdirectories `class_a` and `class_b`, together with labels
  0 and 1 (0 corresponding to `class_a` and 1 corresponding to `class_b`).

  Only `.txt` files are supported at this time.

  Arguments:
    directory: Directory where the data is located.
        If `labels` is "inferred", it should contain
        subdirectories, each containing text files for a class.
        Otherwise, the directory structure is ignored.
    labels: Either "inferred"
        (labels are generated from the directory structure),
        or a list/tuple of integer labels of the same size as the number of
        text files found in the directory. Labels should be sorted according
        to the alphanumeric order of the text file paths
        (obtained via `os.walk(directory)` in Python).
    label_mode:
        - 'int': means that the labels are encoded as integers
            (e.g. for `sparse_categorical_crossentropy` loss).
        - 'categorical' means that the labels are
            encoded as a categorical vector
            (e.g. for `categorical_crossentropy` loss).
        - 'binary' means that the labels (there can be only 2)
            are encoded as `float32` scalars with values 0 or 1
            (e.g. for `binary_crossentropy`).
        - None (no labels).
    class_names: Only valid if "labels" is "inferred". This is the explict
        list of class names (must match names of subdirectories). Used
        to control the order of the classes
        (otherwise alphanumerical order is used).
    batch_size: Size of the batches of data. Default: 32.
    max_length: Maximum size of a text string. Texts longer than this will
      be truncated to `max_length`.
    shuffle: Whether to shuffle the data. Default: True.
        If set to False, sorts the data in alphanumeric order.
    seed: Optional random seed for shuffling and transformations.
    validation_split: Optional float between 0 and 1,
        fraction of data to reserve for validation.
    subset: One of "training" or "validation".
        Only used if `validation_split` is set.
    follow_links: Whether to visits subdirectories pointed to by symlinks.
        Defaults to False.

  Returns:
    A `tf.data.Dataset` object.
      - If `label_mode` is None, it yields `string` tensors of shape
        `(batch_size,)`, containing the contents of a batch of text files.
      - Otherwise, it yields a tuple `(texts, labels)`, where `texts`
        has shape `(batch_size,)` and `labels` follows the format described
        below.

  Rules regarding labels format:
    - if `label_mode` is `int`, the labels are an `int32` tensor of shape
      `(batch_size,)`.
    - if `label_mode` is `binary`, the labels are a `float32` tensor of
      1s and 0s of shape `(batch_size, 1)`.
    - if `label_mode` is `categorial`, the labels are a `float32` tensor
      of shape `(batch_size, num_classes)`, representing a one-hot
      encoding of the class index.
  """
  if labels != 'inferred':
    if not isinstance(labels, (list, tuple)):
      raise ValueError(
          '`labels` argument should be a list/tuple of integer labels, of '
          'the same size as the number of text files in the target '
          'directory. If you wish to infer the labels from the subdirectory '
          'names in the target directory, pass `labels="inferred"`. '
          'If you wish to get a dataset that only contains text samples '
          '(no labels), pass `labels=None`.')
    if class_names:
      raise ValueError('You can only pass `class_names` if the labels are '
                       'inferred from the subdirectory names in the target '
                       'directory (`labels="inferred"`).')
  if label_mode not in {'int', 'categorical', 'binary', None}:
    raise ValueError(
        '`label_mode` argument must be one of "int", "categorical", "binary", '
        'or None. Received: %s' % (label_mode,))
  dataset_utils.check_validation_split_arg(
      validation_split, subset, shuffle, seed)

  if seed is None:
    seed = np.random.randint(1e6)
  file_paths, labels, class_names = dataset_utils.index_directory(
      directory,
      labels,
      formats=('.txt',),
      class_names=class_names,
      shuffle=shuffle,
      seed=seed,
      follow_links=follow_links)

  if label_mode == 'binary' and len(class_names) != 2:
    raise ValueError(
        'When passing `label_mode="binary", there must exactly 2 classes. '
        'Found the following classes: %s' % (class_names,))

  file_paths, labels = dataset_utils.get_training_or_validation_split(
      file_paths, labels, validation_split, subset)

  dataset = paths_and_labels_to_dataset(
      file_paths=file_paths,
      labels=labels,
      label_mode=label_mode,
      num_classes=len(class_names),
      max_length=max_length)
  if shuffle:
    # Shuffle locally at each iteration
    dataset = dataset.shuffle(buffer_size=batch_size * 8, seed=seed)
  dataset = dataset.batch(batch_size)
  # Users may need to reference `class_names`.
  dataset.class_names = class_names
  return dataset


def paths_and_labels_to_dataset(file_paths,
                                labels,
                                label_mode,
                                num_classes,
                                max_length):
  """Constructs a dataset of text strings and labels."""
  path_ds = dataset_ops.Dataset.from_tensor_slices(file_paths)
  string_ds = path_ds.map(
      lambda x: path_to_string_content(x, max_length))
  if label_mode:
    label_ds = dataset_utils.labels_to_dataset(labels, label_mode, num_classes)
    string_ds = dataset_ops.Dataset.zip((string_ds, label_ds))
  return string_ds


def path_to_string_content(path, max_length):
  txt = io_ops.read_file(path)
  if max_length is not None:
    txt = string_ops.substr(txt, 0, max_length)
  return txt

In [86]:
data = text_dataset_from_directory(directory, labels='inferred')

Found 8082 files belonging to 2 classes.
Using 1616 files for validation.


In [87]:
# Example from here: https://www.tensorflow.org/tutorials/load_data/text

In [88]:
import collections
import pathlib
import re
import string

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import tensorflow_datasets as tfds

batch_size = 32
seed = 42

raw_train_ds = preprocessing.text_dataset_from_directory(
    directory,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

Found 8082 files belonging to 2 classes.
Using 6466 files for training.


In [89]:
#pip install https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.8.0-py3-none-any.whl

In [90]:
raw_val_ds = preprocessing.text_dataset_from_directory(
    directory,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

Found 8082 files belonging to 2 classes.
Using 1616 files for validation.


In [91]:
test_dir = directory
raw_test_ds = preprocessing.text_dataset_from_directory(
    test_dir, batch_size=batch_size)

Found 8082 files belonging to 2 classes.


In [92]:
VOCAB_SIZE = 10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

In [93]:
MAX_SEQUENCE_LENGTH = 30

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

In [94]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

In [95]:
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label

In [96]:
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return binary_vectorize_layer(text), label

In [97]:
# Retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print("Review", first_question)
print("Label", first_label)

Review tf.Tensor(b'A sun-drenched masterpiece, part parlor game, part psychological case study, part droll social satire.', shape=(), dtype=string)
Label tf.Tensor(1, shape=(), dtype=int32)


In [98]:
print("'binary' vectorized question:", 
      binary_vectorize_text(first_question, first_label)[0])

'binary' vectorized question: tf.Tensor([[1. 0. 1. ... 0. 0. 0.]], shape=(1, 10000), dtype=float32)


In [99]:
print("'int' vectorized question:",
      int_vectorize_text(first_question, first_label)[0])

'int' vectorized question: tf.Tensor(
[[   3 8704  685  215    1  539  215  555  569  369  215 2100  581  642
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]], shape=(1, 30), dtype=int64)


In [100]:
print("338 ---> ", int_vectorize_layer.get_vocabulary()[338])
print("299 ---> ", int_vectorize_layer.get_vocabulary()[299])
print("Vocabulary size: {}".format(len(int_vectorize_layer.get_vocabulary())))

338 --->  especially
299 --->  under
Vocabulary size: 10000


In [101]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [102]:
binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_val_ds = raw_val_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)

In [103]:
binary_model = tf.keras.Sequential([layers.Dense(4)])
binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = binary_model.fit(
    binary_train_ds, validation_data=binary_val_ds, epochs=10)

Epoch 1/10
203/203 [==============================] - 5s 21ms/step - loss: 1.0992 - accuracy: 0.6029 - val_loss: 0.7110 - val_accuracy: 0.6850
Epoch 2/10
203/203 [==============================] - 4s 18ms/step - loss: 0.6671 - accuracy: 0.7030 - val_loss: 0.6056 - val_accuracy: 0.7450
Epoch 3/10
203/203 [==============================] - 4s 18ms/step - loss: 0.5483 - accuracy: 0.7981 - val_loss: 0.5500 - val_accuracy: 0.7865
Epoch 4/10
203/203 [==============================] - 4s 19ms/step - loss: 0.4750 - accuracy: 0.8674 - val_loss: 0.5131 - val_accuracy: 0.8106
Epoch 5/10
203/203 [==============================] - 4s 19ms/step - loss: 0.4200 - accuracy: 0.8987 - val_loss: 0.4860 - val_accuracy: 0.8205
Epoch 6/10
203/203 [==============================] - 4s 18ms/step - loss: 0.3781 - accuracy: 0.9221 - val_loss: 0.4645 - val_accuracy: 0.8261
Epoch 7/10
203/203 [==============================] - 4s 18ms/step - loss: 0.3420 - accuracy: 0.9346 - val_loss: 0.4471 - val_accuracy: 0.8304

In [104]:
def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  return model

In [105]:
# vocab_size is VOCAB_SIZE + 1 since 0 is used additionally for padding.
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=4)
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=5)

Epoch 1/5
203/203 [==============================] - 5s 22ms/step - loss: 0.8698 - accuracy: 0.6432 - val_loss: 0.5910 - val_accuracy: 0.6751
Epoch 2/5
203/203 [==============================] - 5s 22ms/step - loss: 0.5018 - accuracy: 0.7471 - val_loss: 0.3679 - val_accuracy: 0.8391
Epoch 3/5
203/203 [==============================] - 5s 22ms/step - loss: 0.1875 - accuracy: 0.9381 - val_loss: 0.3528 - val_accuracy: 0.8441
Epoch 4/5
203/203 [==============================] - 5s 22ms/step - loss: 0.0516 - accuracy: 0.9886 - val_loss: 0.3663 - val_accuracy: 0.8546
Epoch 5/5
203/203 [==============================] - 5s 22ms/step - loss: 0.0124 - accuracy: 0.9987 - val_loss: 0.4001 - val_accuracy: 0.8490


In [106]:
print("Linear model on binary vectorized data:")
print(binary_model.summary())

Linear model on binary vectorized data:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 40004     
Total params: 40,004
Trainable params: 40,004
Non-trainable params: 0
_________________________________________________________________
None


In [107]:
print("ConvNet model on int vectorized data:")
print(int_model.summary())

ConvNet model on int vectorized data:
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          640064    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          20544     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 660,868
Trainable params: 660,868
Non-trainable params: 0
_________________________________________________________________
None


In [108]:
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_ds)
int_loss, int_accuracy = int_model.evaluate(int_test_ds)

print("Binary model accuracy: {:2.2%}".format(binary_accuracy))
print("Int model accuracy: {:2.2%}".format(int_accuracy))

253/253 [==============================] - 4s 14ms/step - loss: 0.0833 - accuracy: 0.9698
Binary model accuracy: 94.91%
Int model accuracy: 96.98%


In [109]:
export_model = tf.keras.Sequential(
    [binary_vectorize_layer, binary_model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print("Accuracy: {:2.2%}".format(binary_accuracy))

253/253 [==============================] - 4s 14ms/step - loss: 0.2764 - accuracy: 0.9465
Accuracy: 94.91%


In [110]:
def get_string_labels(predicted_scores_batch):
  predicted_int_labels = tf.argmax(predicted_scores_batch, axis=1)
  predicted_labels = tf.gather(raw_train_ds.class_names, predicted_int_labels)
  return predicted_labels

In [126]:
inputs = [
    "covid 19 was the best thing that ever happend to me",  # pos
    "Death is everywhere",  # neg
    "loved it from start to finish",  # pos
    "covid 19 killed my cat",  # neg
    "trump is full of hatred and killing",  # neg
    "trump is full of hatred and killing",  # neg
]
predicted_scores = export_model.predict(inputs)
predicted_labels = get_string_labels(predicted_scores)
for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("Predicted label: ", label.numpy())

Question:  covid 19 was the best thing that ever happend to me
Predicted label:  b'pos'
Question:  Death is everywhere
Predicted label:  b'neg'
Question:  loved it from start to finish
Predicted label:  b'pos'
Question:  covid 19 killed my cat
Predicted label:  b'neg'
Question:  trump is full of hatred and killing
Predicted label:  b'neg'
Question:  trump is full of hatred and killing
Predicted label:  b'neg'
